Instalação do postgres

In [1]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS enterprise;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE enterprise;'

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 11.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../01-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Preparing to unpack .../03-libpq

In [2]:
%env TFIO_DEMO_DATABASE_NAME=enterprise
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

env: TFIO_DEMO_DATABASE_NAME=enterprise
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=postgres
env: TFIO_DEMO_DATABASE_PASS=postgres


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f '/content/northwind.sql'
!psql postgres < '/content/drive/Shareddrives/modulo4/Tarefa 1: Aula Prática 1 (SQL)/northwind.sql'

/content/northwind.sql: No such file or directory
SET
SET
SET
SET
SET
SET
SET
SET
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1

In [5]:
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [6]:
# Compras dos usuários por país (Agregação e junção)
df_sales_country = % sql select customers.country, count(orders.order_id) as nro_vendas, round(sum(order_details.unit_price * order_details.quantity) - sum(order_details.unit_price * order_details.quantity * order_details.discount)) as U$_total \
from customers \
inner join orders on orders.customer_id = customers.customer_id \
inner join order_details on orders.order_id = order_details.order_id \
group by customers.country \
order by U$_total desc

df_sales_country


 * postgresql+psycopg2://@/postgres


,country,nro_vendas,u$_total
0,USA,352,245585.0
1,Germany,328,230285.0
2,Austria,125,128004.0
3,Brazil,203,106926.0
4,France,184,81358.0
5,UK,135,58971.0
6,Venezuela,118,56811.0
7,Sweden,97,54495.0
8,Canada,75,50196.0
9,Ireland,55,49980.0


In [7]:
# Categorização de usuário por quantidade pedidos (Classificação)
df_user_cat_by_purchases = %sql Select CASE WHEN qnt >= 15 THEN 'Super' WHEN qnt >= 5 and qnt < 15 THEN 'Padrão' ELSE  'Basic' END AS level, customers.* \
from customers join (Select customer_id, count(customer_id) as qnt from orders group by customer_id) as customer_orders on customers.customer_id = customer_orders.customer_id

df_user_cat_by_purchases

 * postgresql+psycopg2://@/postgres


,level,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,Padrão,TOMSP,Toms Spezialitäten,Karin Josephs,Marketing Manager,Luisenstr. 48,Münster,None,44087,Germany,0251-031259,0251-035695
1,Padrão,OLDWO,Old World Delicatessen,Rene Phillips,Sales Representative,2743 Bering St.,Anchorage,AK,99508,USA,(907) 555-7584,(907) 555-2880
2,Padrão,MAGAA,Magazzini Alimentari Riuniti,Giovanni Rovelli,Marketing Manager,Via Ludovico il Moro 22,Bergamo,None,24100,Italy,035-640230,035-640231
3,Padrão,QUEEN,Queen Cozinha,Lúcia Carvalho,Marketing Assistant,"Alameda dos Canàrios, 891",Sao Paulo,SP,05487-020,Brazil,(11) 555-1189,None
4,Padrão,VINET,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11
...,...,...,...,...,...,...,...,...,...,...,...,...
84,Padrão,REGGC,Reggiani Caseifici,Maurizio Moroni,Sales Associate,Strada Provinciale 124,Reggio Emilia,None,42100,Italy,0522-556721,0522-556722
85,Padrão,QUEDE,Que Delícia,Bernardo Batista,Accounting Manager,"Rua da Panificadora, 12",Rio de Janeiro,RJ,02389-673,Brazil,(21) 555-4252,(21) 555-4545
86,Padrão,LAMAI,La maison d'Asie,Annette Roulet,Sales Manager,1 rue Alsace-Lorraine,Toulouse,None,31000,France,61.77.61.10,61.77.61.11
87,Padrão,WANDK,Die Wandernde Kuh,Rita Müller,Sales Representative,Adenauerallee 900,Stuttgart,None,70563,Germany,0711-020361,0711-035428


In [8]:
# Total de vendas por cidade de destino (lines)
total_purchases_by_dest_city = %sql SELECT ship_city, SUM((unit_price * quantity) - (unit_price * quantity * discount)) AS total_de_vendas from order_details \
            JOIN orders ON order_details.order_id = orders.order_id \
            GROUP BY ship_city \
            ORDER BY total_de_vendas DESC;

total_purchases_by_dest_city

 * postgresql+psycopg2://@/postgres


,ship_city,total_de_vendas
0,Cunewalde,110277.305030
1,Graz,104874.978144
2,Boise,104361.949540
3,Rio de Janeiro,51956.979901
4,Albuquerque,51097.800828
...,...,...
65,Caracas,1488.699997
66,Reims,1480.000002
67,Barcelona,836.699995
68,Vancouver,522.499992


In [9]:
# Gerar csv dos processos de ETL
df_sales_country.to_csv('df_sales_country.csv', sep = '|')
df_user_cat_by_purchases.to_csv('df_user_cat_by_purchases.csv', sep = '|')
total_purchases_by_dest_city.to_csv('total_purchases_by_dest_city.csv', sep = '|')
